In [273]:
import pandas as pd
from utils.sheet_operator import expenses_sheet
from utils.gcp_operator import gcs_writer

In [274]:
df = expenses_sheet()

In [275]:
# ajustando a estrutura do DataFrame, removendo o cabeçalho e trazendo a 1a linha para as colunas
df.columns = df.iloc[0]
df = df[1:].reset_index().drop('index', axis=1)

In [276]:
# criando um dicionário para as categorias da planilha
categories_indices = df.index[df['filter'] == '1'].tolist()
categories_dict = dict()

for index in enumerate(categories_indices):
    next_index = df.index[-1] if index[0] == len(categories_indices)-1 else categories_indices[index[0]+1]
    category = df.iloc[index[1]]['Mês']
    categories_dict[category] = df.iloc[index[1]+1:next_index]['Mês'].tolist()

In [277]:
# removendo as categorias do DataFrame
df = df[df['filter'] != '1']
df = df.drop('filter', axis=1)

In [278]:
# ajustando a estrutura do DataFrame
df = df.transpose().reset_index()
df.columns = df.iloc[0]
df = df[1:].reset_index().drop('index', axis=1)

In [279]:
# ajustando as colunas de valores de gastos
for column in df.columns:
    if column != 'Mês':
        df[column] = df[column].str.strip().str.replace(r'^R\$', '', regex=True)
        df[column] = df[column].str.replace(r'\s+', '', regex=True)
        df[column] = df[column].str.replace(r'\.', '', regex=True)
        df[column] = df[column].str.replace(r',', '.', regex=True)
        df[column] = df[column].astype(float)

In [280]:
df['Mês'] = pd.to_datetime(df['Mês'])

/tmp/ipykernel_49479/3781474281.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Mês'] = pd.to_datetime(df['Mês'])


In [281]:
for i in categories_dict.keys():
    print(i)

DESPESAS COM PESSOAL
MENSAIS VARIÁVEIS - COBRADOS NO BOLETO CONDOMÍNIO
MENSAIS FIXO
MANUTENÇÃO E CONSERVAÇÃO - (Prováveis de ocorrer*)
DESPESAS DIVERSAS
DESPESAS ADMINISTRATIVAS


In [282]:
# criando dicionário para renomear colunas
rename_columns = {
    'Mês': 'mes',
    # DESPESAS COM PESSOAL
    'Total com Salários': 'total_salarios',
    'Total com Adiantamento salarial (sempre dia 20)': 'total_adiantamento_salarial',
    'Exame médico periódico': 'exame_medico_periodico',
    'Férias Vitamar': 'ferias_vitamar',
    'Férias Daniel': 'ferias_daniel',
    'Férias Fábio': 'ferias_fabio',
    'Férias Jailza': 'ferias_jailza',
    'Férias Joana': 'ferias_joana',
    'Férias Karla': 'ferias_karla',
    'Total com Décimo terceiro salário (1 parc - setembro)': 'decimo_terceiro_parcela_1',
    'Total com Décimo terceiro salário (2 parc - dezembro)': 'decimo_terceiro_parcela_2',
    'Ticket alimentação': 'ticket_alimentacao',
    'INSS': 'inss',
    'FGTS': 'fgts',
    'Pis sobre folha de pagamento': 'pis_folha_pagamento',
    'Vale Transporte': 'vale_transporte',
    'Seguro de vida dos funcionários': 'seguro_vida',
    'Uniformes': 'uniformes'
}

In [283]:
# renomeando colunas
columns = categories_dict['DESPESAS COM PESSOAL']
columns.insert(0, 'Mês')
df = df[columns].rename(columns=rename_columns)

In [286]:
gcs_writer(df, 'extensao-cond', 'curated/expenses/people.csv')